In [1]:
import pickle
import numpy as np
import pandas as pd
import xarray as xr
import panel as pn
import structure

In [2]:
# from importlib import reload
# reload(structure)

In [3]:
pn.extension('vega')
pn.extension('mathjax')

In [4]:
df_pressure = np.abs(structure.q.sel(cat_prefix=['N'],wind_cat=['N1', 'N2', 'N3', 'N4', 'N5', 'N6'],structure='Wall')).max(['t','direction','s_edge']).to_dataframe().unstack(level=0)
df_pressure = df_pressure.droplevel(level=0,axis=0).drop('structure',axis=1).droplevel(level=0,axis=1).round()/1000

In [5]:
a_input, b_input = [pn.widgets.EditableIntSlider(
    start=100,end=5000,value=1000,name=name
) for name in ['a','b']]

In [6]:
df_lh = pd.concat({
    #('wind speed','m/s'):structure.wind_speed.loc['N'],
    ('wind speed','km/h'):structure.wind_speed.loc['N']*3.6,
    ('wind pressure','kPa'):df_pressure},
    names=['variable','unit']
).unstack(level=[0,1]).stack(level=0)

@pn.depends(a=a_input,b=b_input)
def panel_properties(a,b):
    area = a*b/1e6
    aratio = np.round(a/b,2)
    span60 = np.min([a,b])/60
    # return pn.pane.Mar(
    #     pn.widgets.StaticText(value=area,name='Area (sqm)'),
    #     pn.widgets.StaticText(value=aratio,name='Asepct ratio')
    # )
    return pn.pane.Markdown(f'''
    - **Area**: {area:.2f}m$$^2$$
    - **Aspect ratio**: {aratio:.2f}
    - **Span/60**: {span60:.0f}mm
    ''')

In [7]:
from matplotlib.cm import YlOrRd, Greys, RdYlGn_r
from matplotlib.colors import rgb2hex

In [8]:
def cm_x(cmap=Greys,vmin=0,vmax=1):
    def fn(x):
        return rgb2hex(cmap(int((x*(vmax-vmin)+vmin)*255)))
    return fn

In [24]:
df_lh_ls = df_lh.unstack(level=1).reorder_levels([2,0,1],axis=1)
df_lh_ls.columns = pd.MultiIndex.from_tuples(
    [(a,b,c,'') for a,b,c in df_lh_ls.columns],
    names=['limit state','var','unit','glass thickness']
)

def eq_point_load(a,b):
    '''
    returns equivalent concentrated load (kgf) applied at the center of panel
    that would cause the same deformation as the SLS wind pressure. 
    '''
    xs = xr.Dataset(coords={'x':[a/b]})
    coef = structure.navier_xr(xs.x)['alpha']/structure.navier_pointload_xr(xs.x)['alpha']/xs.x*a*b/1e6
    Feq = (df_pressure['SLS'].to_xarray()*coef*100).round() # 1kN=100kgf
    Feq.name = 'Feq_SLS'
    Feq.attrs = {'unit':'kgf'}
    Feq = Feq.max('x').to_dataframe()
    Feq.columns = pd.MultiIndex.from_tuples([('point load*','kgf','')])
    return Feq

vmin=0.3
@pn.depends(a=a_input,b=b_input)
def wind_deflection_stress(a,b):
    '''
    a - long dimension in mm
    b - short dimension in mm
    pressure in kPa
    force in kgf
    '''
    dfa = structure.wind_pressure_deflection(a,b).sel(
        cat_prefix=['N'],
        wind_cat=['N1', 'N2', 'N3', 'N4', 'N5', 'N6'],
        structure='Wall'
    ).max(
        ['t_spacer','spacer_ratio']
    ).to_dataframe(
    ).droplevel(
        level=[0,1,3]
    )[['Sratio','Smax','yratio_span60','ymax']].unstack(level=0)


    
    def styler(row,cmap=RdYlGn_r,field='Sratio'):
        '''
        Helper function for colour coding ymax and Smax 
        cell backgrounds by yratio and Sratio
        
        Toggles text colour between black and white to improve readability
        '''
        fn = cm_x(cmap,vmin=vmin,vmax=1)
        return [f"background-color: {fn(x)}; color:{'black' if x<0.75 else 'white'}" 
                for x in dfa[field].loc[row.name,row.index.get_level_values(2)]]

    df_ls = {}
    for limit_state, var, ratio, precision, unit in [
        ('SLS','ymax','yratio_span60','{:.0f}','mm'),
        ('SLS','yratio_span60','yratio_span60','{:.2f}','1'),
        ('ULS','Smax','Sratio','{:.0f}','MPa'),
        ('ULS','Sratio','Sratio','{:.2f}','1'),
    ]:
        df_rh = dfa[var]
        df_rh.columns = pd.MultiIndex.from_tuples([(var,unit,x) for x in df_rh.columns],names=['var','unit','glass thickness'])
        if limit_state=='SLS':
            df_lh = df_lh_ls[limit_state].join(eq_point_load(a,b))
            varlist = ['wind speed','point load*']
        else:
            df_lh = df_lh_ls[limit_state]
            varlist = ['wind speed']
        df_tmp = df_lh.join(df_rh)
        df_ls[limit_state,var] = df_tmp.style.apply(
            styler,field=ratio,subset=[var],axis=1
        ).format(
            precision,subset=[var]
        ).format(
            '{:.0f}',subset=varlist
        ).format(
            '{:.2f}',subset=['wind pressure']
        )
    #return pn.pane.DataFrame(df_lh,width=300)
    return pn.Tabs(*[(' '.join(k),
                      pn.pane.DataFrame(v,min_width=450,sizing_mode='fixed')) 
                     for k,v in df_ls.items()],
                  min_width=450,sizing_mode='fixed')

# App definition

In [10]:
import altair as alt

In [11]:
xs = np.linspace(0,1,6)
colors = [cm_x(RdYlGn_r,vmin=vmin)(x) for x in xs]
df_tmp = pd.DataFrame({'x':xs,'color':colors})
colorbar = alt.Chart(df_tmp).mark_rect().encode(
    x=alt.X('x:N',axis=alt.Axis(format='.1f',title='',labelAngle=0)),
    color=alt.Color('color',scale=None)
).properties(
    width=180,
    height=30
)

In [12]:
colorbar

alt.Chart(...)

In [25]:
app = pn.Column(
    pn.pane.Markdown('# DGU Wind Deflection Calculator'),
    pn.Row(
    pn.Column(
        pn.pane.Markdown(r'''
        ## 1. Overview
        - Computes Navier solution for deflection of rectangular flat plates
        - 4 edge simple support
        - Uniformly distributed load (wind pressure in kPa)
        - Load-sharing factor of 0.625 is applied in accordance with AS1288-2021 3.4.2 
        $$k_{pane} = \frac{1.25t^3_{pane}}{\Sigma_i t_i^3} \le 1$$

        ## 2. Usage instruction
        - Type or adjust sliders a & b to the target dimension of the DGU in mm
        - Returns predicted panel deflection in mm
        - Allow 1-2 sec for table to update
        - Click on the Tab headers to check predictions for:
            - **ymax**: maximum predicted deflection (mm)
            - **yratio_span60**: predicted deflection/allowable deflection (span/60)
            - **Smax**: maximum predicted stress (MPa)
            - **Sratio**: predicted stress/stress capacity for monlithic toughened glass
        - Color coding is based on capacity ratios (y or S)
        '''),
        pn.pane.Vega(colorbar,align='center'),
        width_policy='min',
        min_width=350
    ),
    pn.Column(
        pn.pane.Markdown('''
        ## 3. Predictions
        '''),
        a_input, 
        b_input,
        panel_properties,
        wind_deflection_stress,
        width_policy='min'
    ),
    pn.pane.Markdown(f'''
    ## 4. Glossary
    - __wind pressure__: Serviceability limit state design wind pressure (kPa) for housing
        - Derived from AS1288-2021 Table A.2, which is based on the methodology documented in AS4055
        - The values displayed differs from AS1288-2021 Table A.2 due to the use of equal pane thickness 
    IGU load-sharing factor. Divide by ({structure.k_pane_eq}) to retrieve the original pressure value.
    - __point load*__: equivalent concentrated load applied at the center of panel that 
    would cause the same deformation as the wind pressure. 
        - Displayed in unit of __kilogram force__(kgf), here defined as 1kgf=10N, approximately the force exerted by gravity on 1kg mass.
        - Whereas wind pressure is independent of panel size, the equivalent point load depend on both the aspect ratio and total panel area.
    - __Limit States__:
        - Serviceability Limit State (SLS): Deflection limit
        - Ultimate Limit State (ULS): Strength limit
    ''',
        width_policy='min',
        min_width=400
    )
)
)

In [26]:
app.servable()

Column
    [0] Markdown(str)
    [1] Row
        [0] Column(min_width=350, width_policy='min')
            [0] Markdown(str)
            [1] Vega(Chart, align='center', selection=Selection)
        [1] Column(width_policy='min')
            [0] Markdown(str)
            [1] EditableIntSlider(end=5000, name='a', start=100, value=1756, width=300)
            [2] EditableIntSlider(end=5000, name='b', start=100, value=1871, width=300)
            [3] ParamFunction(function, _pane=Markdown, defer_load=False)
            [4] ParamFunction(function, _pane=Tabs, defer_load=False)
        [2] Markdown(str, min_width=400, width_policy='min')

In [28]:
# server = app.show(
#     websocket_origin='fgstopsolid:64092',
#     address='fgstopsolid',
#     port=64092
# )

Launching server at http://fgstopsolid:64092


In [27]:
# server.stop()